In [2]:
import findspark
findspark.init()

In [3]:
from pyspark.sql import SparkSession

spark1 = SparkSession.builder.appName("PracticeApp").getOrCreate()

print("Spark Session for Practice is Created")
print("Spark version: ", spark1.version)
print("------------- Happy Codding ----------")

Spark Session for Practice is Created
Spark version:  4.0.1
------------- Happy Codding ----------


In [13]:
data = [
    {"name":"masud rana", "cgpa":3.60, "address":"Kanchan, Rupganj, Narayanganj", "university":"Green University of Bangladesh"}
]

df = spark1.createDataFrame(data)
df.show()


+--------------------+----+----------+--------------------+
|             address|cgpa|      name|          university|
+--------------------+----+----------+--------------------+
|Kanchan, Rupganj,...| 3.6|masud rana|Green University ...|
+--------------------+----+----------+--------------------+

